<a href="https://colab.research.google.com/github/samarth70/Analysis-of-Portugese-banking-dataset/blob/master/samarth70/Neural_Style_Transfer_with_gradio_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import IPython.display as display
import matplotlib as mpl
import os
import time
from PIL import Image
import numpy as np
import PIL.Image
import time
import functools
import cv2

In [ ]:
!mkdir nstmodel 
!wget -c https://storage.googleapis.com/tfhub-modules/google/magenta/arbitrary-image-stylization-v1-256/2.tar.gz -O - | tar -xz -C /content/nstmodel


--2021-02-10 13:17:44--  https://storage.googleapis.com/tfhub-modules/google/magenta/arbitrary-image-stylization-v1-256/2.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 2607:f8b0:400e:c08::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85627349 (82M) [application/x-tar]
Saving to: ‘STDOUT’

-                   100%[===================>]  81.66M  76.6MB/s    in 1.1s    

2021-02-10 13:17:45 (76.6 MB/s) - written to stdout [85627349/85627349]



In [ ]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('/content/nstmodel')

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)


In [ ]:
def save_image(image, filename):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save to.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.jpg" % filename)
  print("Saved as %s.jpg" % filename)


In [ ]:
import cv2

def gray_scaled(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img2 = np.zeros_like(img)
  img2[:,:,0] = gray
  img2[:,:,1] = gray
  img2[:,:,2] = gray
  return img2
  

In [ ]:
def transform_mymodel(content_image,style_image):
  # content_image = plt.imread(content_image_path)
  # style_image = plt.imread(style_image_path)
  # Convert to float32 numpy array, add batch dimension, and normalize to range [0, 1]
  # content_image=gray_scaled(content_image)
  content_image = content_image.astype(np.float32)[np.newaxis, ...] / 255.0
  style_image = style_image.astype(np.float32)[np.newaxis, ...] / 255.0
  # Optionally resize the images. It is recommended that the style image is about
  # 256 pixels (this size was used when training the style transfer network).
  # The content image can be any size.
  style_image = tf.image.resize(style_image, (256, 256))

  # Stylize image.
  outputs = model(tf.constant(content_image), tf.constant(style_image))
  stylized_image = outputs[0]
  # stylized = tf.image.resize(stylized_image, (356, 356))
  stylized_image =tensor_to_image(stylized_image)
  save_image(stylized_image,'stylized')
  return stylized_image


In [ ]:
def gradio_intrface(mymodel):
# initializing the input component 
  image1 = gr.inputs.Image() 
  image2 = gr.inputs.Image() 
  stylizedimg=gr.outputs.Image()
  gr.Interface(fn=mymodel, inputs= [image1,image2] , outputs= stylizedimg,title='Style Transfer').launch(share=False,)

In [ ]:
gradio_intrface(transform_mymodel)